In [3]:
import json
import os
import pandas as pd
import pickle

from utils.data_dirs import data_dirs
from utils.index import index
from utils.unique import unique

In [4]:
EXPERIMENT = False

In [5]:
base_dir, output_dir, folder, EXPERIMENT_DATA, suffix, *_ = data_dirs(EXPERIMENT)
print(base_dir)
print(output_dir)
print(folder)
print(EXPERIMENT_DATA)
print(suffix)

/work/apperception/data/raw/nuScenes/full-dataset-v1.0/Mini
/work/apperception/data/processed/nuScenes/full-dataset-v1.0/Mini
v1.0-mini
/work/apperception/data/raw/scenic/experiment_data



In [43]:
with open(os.path.join(base_dir, folder, 'category.json')) as f:
    category_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample.json')) as f:
    sample_json = json.load(f)

with open(os.path.join(base_dir, folder, 'sample_annotation.json')) as f:
    sample_annotation_json = json.load(f)

with open(os.path.join(base_dir, folder, 'instance.json')) as f:
    instance_json = json.load(f)

with open(os.path.join(base_dir, folder, 'scene.json')) as f:
    scene_json = json.load(f)

with open(os.path.join(base_dir, folder, 'attribute.json')) as f:
    attribute_json = json.load(f)

with open(os.path.join(base_dir, folder, 'log.json')) as f:
    log_json = json.load(f)

In [48]:
log_filter = [
    {
        "log_token": l["token"],
        "location": l["location"]
    }
    for l in log_json
]
len(log_filter)

8

In [49]:
pd.DataFrame(log_filter)

,log_token,location
0,7e25a2c8ea1f41c5b0da1e69ecfa71a2,singapore-onenorth
1,53cf9c55dd8644bea67b9f009fc1ee38,boston-seaport
2,6f7fe59adf984e55a82571ab4f17e4e2,boston-seaport
3,881dd2f8aaec49b681eb54be5bf3b3e2,boston-seaport
4,3a43824b84534c98bda1d07548db5817,boston-seaport
5,8ff48ad1df8e4966a2151730c92b7f3c,singapore-queenstown
6,5bd40b613ac740cd9dbacdfbc3d68201,singapore-queenstown
7,8fefc430cbfa4c2191978c0df302eb98,singapore-hollandvillage


In [50]:
sample_filter = [
    {
        "sample_token": s["token"],
        "timestamp": s["timestamp"],
        "scene_token": s["scene_token"]
    }
    for s in sample_json
]
len(sample_filter)

404

In [51]:
pd.DataFrame(sample_filter)

,sample_token,timestamp,scene_token
0,ca9a282c9e77460f8360f564131a8af5,1532402927647951,cc8c0bf57f984915a77078b10eb33198
1,39586f9d59004284a7114a68825e8eec,1532402928147847,cc8c0bf57f984915a77078b10eb33198
2,356d81f38dd9473ba590f39e266f54e5,1532402928698048,cc8c0bf57f984915a77078b10eb33198
3,e0845f5322254dafadbbed75aaa07969,1532402929197353,cc8c0bf57f984915a77078b10eb33198
4,c923fe08b2ff4e27975d2bf30934383b,1532402929697797,cc8c0bf57f984915a77078b10eb33198
...,...,...,...
399,8cc06390249d4895b5b9b860f0952f24,1542801005447229,e233467e827140efa4b42d2b4c435855
400,d7cb9aa06de1442d8e2a22d562045cb4,1542801005947122,e233467e827140efa4b42d2b4c435855
401,4d4e4c0fd8ec49bdb12788e0a9ecbce3,1542801006446992,e233467e827140efa4b42d2b4c435855
402,0d6668bcb0644bbaaee81c17cc308845,1542801006946864,e233467e827140efa4b42d2b4c435855


In [52]:
scene_filter = [
    {
        "scene_token": s["token"],
        "log_token": s["log_token"],
        "name": s["name"],
    }
    for s in scene_json
]
len(scene_filter)

10

In [53]:
pd.DataFrame(scene_filter)

,scene_token,log_token,name
0,cc8c0bf57f984915a77078b10eb33198,7e25a2c8ea1f41c5b0da1e69ecfa71a2,scene-0061
1,fcbccedd61424f1b85dcbf8f897f9754,53cf9c55dd8644bea67b9f009fc1ee38,scene-0103
2,6f83169d067343658251f72e1dd17dbc,881dd2f8aaec49b681eb54be5bf3b3e2,scene-0553
3,bebf5f5b2a674631ab5c88fd1aa9e87a,6f7fe59adf984e55a82571ab4f17e4e2,scene-0655
4,2fc3753772e241f2ab2cd16a784cc680,3a43824b84534c98bda1d07548db5817,scene-0757
5,c5224b9b454b4ded9b5d2d2634bbda8a,8ff48ad1df8e4966a2151730c92b7f3c,scene-0796
6,325cef682f064c55a255f2625c533b75,5bd40b613ac740cd9dbacdfbc3d68201,scene-0916
7,d25718445d89453381c659b9c8734939,8fefc430cbfa4c2191978c0df302eb98,scene-1077
8,de7d80a1f5fb4c3e82ce8a4f213b450a,8fefc430cbfa4c2191978c0df302eb98,scene-1094
9,e233467e827140efa4b42d2b4c435855,8fefc430cbfa4c2191978c0df302eb98,scene-1100


In [26]:
sample_annotation_filter = [
    sa
    for sa in sample_annotation_json
]
len(sample_annotation_filter)

18538

In [34]:
pd.DataFrame(sample_annotation_filter)

,token,sample_token,instance_token,visibility_token,attribute_tokens,translation,size,rotation,prev,next,num_lidar_pts,num_radar_pts
0,70aecbe9b64f4722ab3c230391a3beb8,cd21dbfc3bd749c7b10a5c42562e0c42,6dd2cbf4c24b4caeb625035869bca7b5,4,[4d8821270b4a47e3a8a300cbec48188e],"[373.214, 1130.48, 1.25]","[0.621, 0.669, 1.642]","[0.9831098797903927, 0.0, 0.0, -0.183016295062...",a1721876c0944cdd92ebc3c75d55d693,1e8e35d365a441a18dd5503a0ee1c208,5,0
1,7fa3a688931b4500b7ce29d187d3b975,2ff86dc19c4644a1a88ce5ba848f56e5,6dd2cbf4c24b4caeb625035869bca7b5,4,[4d8821270b4a47e3a8a300cbec48188e],"[373.152, 1130.357, 1.25]","[0.621, 0.669, 1.642]","[0.9831098797903927, 0.0, 0.0, -0.183016295062...",1e8e35d365a441a18dd5503a0ee1c208,913072e56d6c4025b9b47ba085dd6d7c,6,0
2,93d5b79041c64693a5b32f1103a39a06,29e056fc277543c9bc42310b122c640e,6dd2cbf4c24b4caeb625035869bca7b5,1,[4d8821270b4a47e3a8a300cbec48188e],"[373.35, 1130.67, 1.123]","[0.621, 0.669, 1.642]","[0.9831098797903927, 0.0, 0.0, -0.183016295062...",3b24f083c0bf42d695a1040efdab7ffe,8bb63134d48840aaa2993f490855ff0d,2,0
3,ef63a697930c4b20a6b9791f423351da,ca9a282c9e77460f8360f564131a8af5,6dd2cbf4c24b4caeb625035869bca7b5,1,[4d8821270b4a47e3a8a300cbec48188e],"[373.256, 1130.419, 0.8]","[0.621, 0.669, 1.642]","[0.9831098797903927, 0.0, 0.0, -0.183016295062...",,7987617983634b119e383d8a29607fd7,1,0
4,7987617983634b119e383d8a29607fd7,39586f9d59004284a7114a68825e8eec,6dd2cbf4c24b4caeb625035869bca7b5,1,[4d8821270b4a47e3a8a300cbec48188e],"[373.256, 1130.419, 0.81]","[0.621, 0.669, 1.642]","[0.9831098797903927, 0.0, 0.0, -0.183016295062...",ef63a697930c4b20a6b9791f423351da,9acb7dfed3454f72b2874dda3bdacc48,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
18533,47327ad48c754ae182837e12a9dacac3,c36e8482feb041de9f081e76396cf4fe,88b7a3d6e9394198a1511642a5aa5499,4,[cb5118da1ab342aa947717dc53544259],"[1327.591, 918.656, 1.306]","[1.836, 4.416, 1.43]","[0.9048506452970881, 0.0, 0.0, 0.4257291506409...",b8dd2792bd4a4b1eb4deba5083bf74f4,d78c1081d77e4928a855cde17c171c46,25,4
18534,7261b1ae5c5c4253941f1b3298f108db,b902d29df24c4efda414a9a88ed57031,88b7a3d6e9394198a1511642a5aa5499,1,[cb5118da1ab342aa947717dc53544259],"[1347.981, 943.716, 2.682]","[1.836, 4.416, 1.43]","[0.9048506452970881, 0.0, 0.0, 0.4257291506409...",a0dee55f138f4b46884458b3de7cefa1,,0,1
18535,8fbe30c0e281488f9a2e4a061fc92c53,0a3abc33048d46f9bd78151d1df4b004,88b7a3d6e9394198a1511642a5aa5499,1,[cb5118da1ab342aa947717dc53544259],"[1337.789, 931.193, 2.006]","[1.836, 4.416, 1.43]","[0.9048506452970881, 0.0, 0.0, 0.4257291506409...",324685f934c54ccbab114bd4ec088f60,6d6588ac358743c2bf850c6d0e9c5f56,6,1
18536,fb2873a3231945249b1fe1acf0cedf1d,a480496a5988410fbe3d8ed6c84da996,88b7a3d6e9394198a1511642a5aa5499,4,[cb5118da1ab342aa947717dc53544259],"[1315.676, 902.276, 0.406]","[1.836, 4.416, 1.43]","[0.8854193832467303, 0.0, 0.0, 0.4647929816283...",,59ea3960081c49a4a579e83805f5efe2,659,1


In [27]:
instance_tokens = unique(sample_annotation_filter, 'instance_token')
instance_filter = [
    {
        'instance_token': i['token'],
        'category_token': i['category_token']
    }
    for i in instance_json
    if i['token'] in instance_tokens
]
len(instance_filter)

911

In [32]:
pd.DataFrame(instance_filter)

,instance_token,category_token
0,6dd2cbf4c24b4caeb625035869bca7b5,1fa93b757fc74fb197cdd60001ad8abf
1,48d58b69b40149aeb2e64aa4b1a9192f,1fa93b757fc74fb197cdd60001ad8abf
2,bd26c2cdb22d4bb1834e808c89128898,fd69059b62a3469fbaef25340c0eab7f
3,6e04e175f20d402f8cbbdf701fe99d92,1fa93b757fc74fb197cdd60001ad8abf
4,4a5a657f59914516bade8551072e5ab0,85abebdccd4d46c7be428af5a6173947
...,...,...
906,6b04180e41cf473c92c1bc1059c3bca8,fd69059b62a3469fbaef25340c0eab7f
907,c7925b44db7f4cfdb37e98c29acb8087,fd69059b62a3469fbaef25340c0eab7f
908,1c6ea643f2d04632ac5624e0072aa659,1fa93b757fc74fb197cdd60001ad8abf
909,60e1ec6251134f9a8e2031b871a455d1,1fa93b757fc74fb197cdd60001ad8abf


In [28]:
category_tokens = unique(instance_filter, 'category_token')
category_filter = [
    {
        'category_token': c['token'],
        'category': c['name']
    }
    for c in category_json
    if c['token'] in category_tokens
]
len(category_filter)

18

In [60]:
instance_map = index(instance_filter, 'instance_token')
category_map = index(category_filter, 'category_token')
sample_map = index(sample_filter, 'sample_token')
log_map = index(log_filter, 'log_token')
scene_map = index(scene_filter, 'scene_token')

def sa_map(sa):
    instance = instance_map[sa['instance_token']]
    category = category_map[instance['category_token']]
    sample = sample_map[sa['sample_token']]
    scene = scene_map[sample['scene_token']]
    log = log_map[scene['log_token']]
    ret = {
        **sa,
        **instance,
        **category,
        **sample,
        **scene,
        **log,
    }
    
    del ret['token']
    del ret['visibility_token']
    del ret['attribute_tokens']
    del ret['prev']
    del ret['next']
    del ret['num_lidar_pts']
    del ret['num_radar_pts']
    del ret['category_token']
    del ret['scene_token']
    del ret['log_token']
    
    return ret

sample_annotation_res = [*map(sa_map, sample_annotation_filter)]
len(sample_annotation_res)

18538

In [61]:
sample_annotation_final = pd.DataFrame(sample_annotation_res)
sample_annotation_final[:1]

,sample_token,instance_token,translation,size,rotation,category,timestamp,name,location
0,cd21dbfc3bd749c7b10a5c42562e0c42,6dd2cbf4c24b4caeb625035869bca7b5,"[373.214, 1130.48, 1.25]","[0.621, 0.669, 1.642]","[0.9831098797903927, 0.0, 0.0, -0.183016295062...",human.pedestrian.adult,1532402935197541,scene-0061,singapore-onenorth


In [64]:
locations = {}

for a in sample_annotation_res:
    location = a['location']
    if location not in locations:
        locations[location] = []
    locations[location].append(a)

In [66]:
for location, annotations in locations.items():
    base = os.path.join(output_dir, 'partition', location)
    if not os.path.exists(base):
        os.makedirs(base)
    with open(os.path.join(base, 'ground_truth_annotation.pickle'), "wb") as f:
        pickle.dump(annotations, f)
    with open(os.path.join(base, 'ground_truth_annotation.json'), "w") as f:
        json.dump(annotations, f)